## Preprocess data in estimator

In [ ]:
import os
import os.path as op
import tensorflow as tf

HEIGHT, WIDTH, CHANNELS = 32, 32, 3
# USE MEAN, STD FROM TRAINING SET
mean,std = ...

def serving_input_receiver_fn():

    def normalize(image):
        image = tf.subtract(image, mean)
        image = tf.divide(image, std)
        image = tf.reshape(image, [HEIGHT, WIDTH, CHANNELS])
        print(image.shape)
        return image

    # Run preprocessing function on all images in batch
    input_ph = tf.placeholder(tf.float32, shape=[None, HEIGHT, WIDTH, CHANNELS], name='image_binary')
    images_tensor = tf.map_fn(
        normalize, input_ph, back_prop=False)

    return tf.estimator.export.ServingInputReceiver(
        {'input_layer': images_tensor},  # The key here needs match the name of your model's first layer
        {'image_bytes': input_ph})

## Store estimator 

In [ ]:
version = '001'
export_dir = '../../models/' + version

# Define a path to your trained keras model and load it in as a `tf.keras.models.Model`
# If you just trained your model, you may already have it in memory and can skip the below 2 lines
# model_save_fpath = '/path/to/my_model.h5'
# keras_model = tf.keras.models.load_model(model_save_fpath)

# Create an Estimator object
estimator_save_dir = '../../estimators/' + version
estimator = tf.keras.estimator.model_to_estimator(keras_model_path='../models/model.h5',
                                                  model_dir=estimator_save_dir)

### Copy files to parent folder (tensorflow bug)

In [ ]:
from os import listdir
from os.path import join
from shutil import copyfile
current_path = estimator_save_dir + '/keras'
new_path = estimator_save_dir
onlyfiles = [f for f in listdir(current_path)]
for file in onlyfiles:
    copyfile(join(current_path, file), join(new_path,file))
    

## Save model in serving format

In [ ]:
estimator.export_saved_model(export_dir, serving_input_receiver_fn=serving_input_receiver_fn)